### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

### I-1. Find the names of FT

Here, we are cheating, since all the features flags are in the flags.go file

But we have to think on how to automate this step!

Given a software with feature flags, how to extract the feature flags, their names, their expiration date, and so on...

In [3]:
s = ""
with open("./juju-develop/feature/flags.go", "r") as f:
    s+=f.read()+"\n"
print(s)

// Copyright 2015 Canonical Ltd.
// Licensed under the AGPLv3, see LICENCE file for details.

// Package feature package defines the names of the current feature flags.
package feature

// TODO (anastasiamac 2015-03-02)
// Features that have commands that can be blocked,
// command list for "juju block" and "juju unblock"
// needs to be maintained until we can dynamically discover
// these commands.

// LogErrorStack is a developer feature flag to have the LoggedErrorStack
// function in the utils package write out the error stack as defined by the
// errors package to the logger.  The ability to log the error stack is very
// useful in those error cases where you really don't expect there to be a
// failure.  This means that the developers with this flag set will see the
// stack trace in the log output, but normal deployments never will.
const LogErrorStack = "log-error-stack"

// LegacyUpstart is used to indicate that the version-based init system
// discovery code (service.VersionI

In [4]:
source = bytes(s, "utf8")
ast = parser.parse(source)

root_node = ast.root_node

print(root_node.type)
print(root_node.start_point)
print(root_node.end_point)
print(root_node.children[1])

source_file
(0, 0)
(64, 0)
<Node kind=comment, start_point=(1, 0), end_point=(1, 59)>


#### List all keywords

In [5]:
for i in range(len(root_node.children)):
    current_node = root_node.children[i]
    for j in range(len(current_node.children)):
        print(source[current_node.start_byte:current_node.end_byte].decode('utf8'))
        #print(current_node)#.child_by_field_name('const_declaration'))
        #if current_node.children[j].type == "const_spec":
        #    if current_node.children[j].is_named:
        #        print(current_node.children[j].)
        

package feature
package feature
const LogErrorStack = "log-error-stack"
const LogErrorStack = "log-error-stack"
const LegacyUpstart = "legacy-upstart"
const LegacyUpstart = "legacy-upstart"
const DeveloperMode = "developer-mode"
const DeveloperMode = "developer-mode"
const StrictMigration = "strict-migration"
const StrictMigration = "strict-migration"
const Branches = "branches"
const Branches = "branches"
const Generations = "generations"
const Generations = "generations"
const RawK8sSpec = "raw-k8s-spec"
const RawK8sSpec = "raw-k8s-spec"
const ActionsV2 = "actions-v2"
const ActionsV2 = "actions-v2"
const Secrets = "secrets"
const Secrets = "secrets"
const AsynchronousCharmDownloads = "async-charm-downloads"
const AsynchronousCharmDownloads = "async-charm-downloads"
const RaftBatchFSM = "raft-batch-fsm"
const RaftBatchFSM = "raft-batch-fsm"
const RaftAPILeases = "raft-api-leases"
const RaftAPILeases = "raft-api-leases"
const CharmAssumes = "charm-assumes"
const CharmAssumes = "charm-a

### Step I-1. results : juju keywords!!! 

In [6]:
keywords = ["LogErrorStack", "LegacyUpstart", "DeveloperMode", "StrictMigration",
            "Branches", "Generations", "RawK8sSpec", "ActionsV2", "Secrets",
            "AsynchronousCharmDownloads", "RaftBatchFSM", "RaftAPILeases", "CharmAssumes"]

### I-2. List files with keywords

#### List all .go files

In [7]:
go_folders = [x[0] for x in os.walk("./juju-develop/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./juju-develop/api/common/charm/charmorigin.go
./juju-develop/api/controller/controller_test.go
./juju-develop/api/instancemutater/instancemutater_test.go
./juju-develop/core/status/status_history.go
./juju-develop/provider/azure/internal/azureauth/package_test.go
./juju-develop/cmd/juju/model/defaults.go
./juju-develop/api/backups/create.go
./juju-develop/state/backups/export_test.go
./juju-develop/apiserver/facades/agent/retrystrategy/retrystrategy_test.go
./juju-develop/api/bundle/client_test.go


In [8]:
len(go_files)

5873

#### List the interesting files, containing at least one keyword with the featureflag 

In [9]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "featureflag" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./juju-develop/apiserver/embeddedcli.go contains :
-> DeveloperMode


./juju-develop/apiserver/logstream.go contains :
-> DeveloperMode


./juju-develop/apiserver/pubsub.go contains :
-> DeveloperMode


./juju-develop/apiserver/logsink/logsink.go contains :
-> DeveloperMode


./juju-develop/provider/manual/environ.go contains :
-> DeveloperMode


./juju-develop/state/migration_export.go contains :
-> StrictMigration


./juju-develop/state/database.go contains :
-> DeveloperMode


./juju-develop/jujuclient/models.go contains :
-> Branches
-> Generations


./juju-develop/caas/kubernetes/provider/operator.go contains :
-> Secrets


./juju-develop/caas/kubernetes/provider/bootstrap.go contains :
-> Secrets


./juju-develop/caas/kubernetes/provider/application/application.go contains :
-> Secrets


./juju-develop/environs/bootstrap/prepare.go contains :
-> Branches
-> Generations


./juju-develop/service/discovery.go contains :
-> LegacyUpstart


./juju-develop/service/discovery_test.go con

In [10]:
len(go_file_interests)

23

## Step 2 : extract AST

#### The use of py-tree-sitter to extract the AST 

In [11]:
s = ""
with open("./juju-develop/feature/flags.go", "r") as f:
    s+=f.read()+"\n"

source = bytes(s, "utf8")
ast = parser.parse(source)

root_node = ast.root_node

tree = Tree()

root_node = ast.root_node

type_nodes = dict()

source = bytes(s, "utf8")
ast = parser.parse(source)

root_node = ast.root_node

def get_name(node):
    return source[node.start_byte:node.end_byte].decode('utf8')

def get_id(node):
    global type_nodes
    node_type = node.type
    if node_type not in type_nodes:
        type_nodes[node_type]=1
    else:
        type_nodes[node_type]+=1
    return node_type+str(type_nodes[node_type])

def process(root_id, node):
    global tree
    node_id = get_id(node)
    node_name = get_name(node)
    tree.create_node(node_name,
                     node_id,
                     parent = root_id)
    if len(node.children) != 0:
        for i in range(len(node.children)):
            process(node_id, node.children[i])

tree.create_node("root", "root")
for i in range(len(root_node.children)):
    process("root", root_node.children[i])

tree.show()

root
├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── 


├── // ActionsV2 enables the next generation actions UX.
├── // AsynchronousCharmDownloads enables support for asynchronous charm downloads.
├── // Branches will allow for model branches functionality to be used.
├── // CharmAssumes instructs Juju to process assumes expressions from charm
├── // Copyright 2015 Canonical Ltd.
├── // DeveloperMode allows access to developer specific commands and behaviour.
├── // Features that have commands that can be blocked,
├── // Generations will allow for model generation functionality to be used.
├── // LegacyUpstart is used to indicate that the version-based init system
├── // Licensed under the AGPLv3, see LICENCE file for details.
├── // LogErrorStack is a developer feature flag to have the LoggedErrorStack
├── // Package feature package defines the names of the current feature flags.
├── // RaftAPILeases will switch all lease store management

## Step 3 - Search Feature Toggles dependencies in the AST

In [38]:

for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./juju-develop/apiserver/embeddedcli.go 

-> cmdErr != nil && featureflag.Enabled(feature.DeveloperMode)


File :             ./juju-develop/apiserver/logstream.go 

-> err != nil && featureflag.Enabled(feature.DeveloperMode)


File :             ./juju-develop/apiserver/pubsub.go 

-> err != nil && featureflag.Enabled(feature.DeveloperMode)


File :             ./juju-develop/apiserver/logsink/logsink.go 

-> err != nil && featureflag.Enabled(feature.DeveloperMode)


File :             ./juju-develop/provider/manual/environ.go 

-> featureflag.Enabled(feature.DeveloperMode)


File :             ./juju-develop/state/migration_export.go 

-> featureflag.Enabled(feature.StrictMigration)


File :             ./juju-develop/state/database.go 

-> featureflag.Enabled(feature.DeveloperMode)


File :             ./juju-develop/jujuclient/models.go 

-> featureflag.Enabled(feature.Branches) || featureflag.Enabled(feature.Generations)
-> featureflag.Enabled(feature.Branches) 

In [13]:
type_nodes

{'comment': 69,
 'package_clause': 1,
 'package': 1,
 'package_identifier': 36,
 '\n': 357,
 'import_declaration': 1,
 'import': 1,
 'import_spec_list': 1,
 '(': 465,
 'import_spec': 44,
 'interpreted_string_literal': 61,
 '"': 122,
 ')': 465,
 'var_declaration': 8,
 'var': 8,
 'var_spec': 8,
 'identifier': 628,
 '=': 14,
 'expression_list': 98,
 'call_expression': 421,
 'selector_expression': 419,
 '.': 451,
 'field_identifier': 458,
 'argument_list': 421,
 'function_declaration': 7,
 'func': 20,
 'parameter_list': 39,
 'block': 48,
 '{': 66,
 ',': 86,
 '}': 66,
 'raw_string_literal': 4,
 'slice_expression': 6,
 '[': 21,
 'int_literal': 22,
 ':': 24,
 ']': 21,
 'const_declaration': 2,
 'const': 2,
 'const_spec': 2,
 'type_conversion_expression': 1,
 'slice_type': 8,
 'type_identifier': 99,
 'escape_sequence': 26,
 'parameter_declaration': 50,
 'return_statement': 28,
 'return': 28,
 'composite_literal': 11,
 'literal_value': 11,
 'keyed_element': 18,
 'type_declaration': 7,
 'type': 7

In [14]:
# Pattern matching -> does not work yet

query = GO_LANGUAGE.query("""
(function_definition
  name: (identifier) @function.def)

(call
  function: (identifier) @function.call)
""")

captures = query.captures(tree.root_node)

